<a href="https://colab.research.google.com/github/AwaisEhsan67/Kyber512-and-NTRU509-comparison/blob/main/NTRU_code_using_Library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/jschanck/ntru.git



Cloning into 'ntru'...
remote: Enumerating objects: 2446, done.
remote: Counting objects: 100% (307/307), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 2446 (delta 204), reused 215 (delta 126), pack-reused 2139 (from 1)
Receiving objects: 100% (2446/2446), 468.51 KiB | 2.68 MiB/s, done.
Resolving deltas: 100% (1714/1714), done.


In [2]:
cd ntru


/content/ntru


In [3]:
%%shell
sudo apt-get update
sudo apt-get install -y unifdef


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,715 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [47.7 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd6

In [4]:
!mkdir -p hps-pruned

In [5]:
%%shell
for f in ref-hps2048509/*.c; do
  unifdef -k \
    -DNTRU_HPS -UNTRU_HRSS \
    -DNTRU_N=509 -DNTRU_Q=2048 \
    "ref-hps2048509/${f##*/}" \
    > "hps-pruned/${f##*/}"
done
cp ref-hps2048509/*.h hps-pruned/
cp ref-common/*.h     hps-pruned/


In [6]:
%%shell
gcc -shared -fPIC \
    -D'CRYPTO_NAMESPACE(name)=name' \
    hps-pruned/*.c \
    -I hps-pruned \
    -o libntru2048509.so


In [7]:
!ls -lh libntru2048509.so

-rwxr-xr-x 1 root root 47K May  7 02:54 libntru2048509.so


In [8]:
!nm -D libntru2048509.so


0000000000001539 T cmov
00000000000015b2 T cpucycles
00000000000015cd T crypto_sort_int32
                 w __cxa_finalize@GLIBC_2.2.5
0000000000003867 T dec
0000000000003752 T enc
                 w __gmon_start__
                 w _ITM_deregisterTMCloneTable
                 w _ITM_registerTMCloneTable
0000000000001d6b T KeccakF1600_StatePermute
00000000000036bd T keypair
                 U open@GLIBC_2.2.5
0000000000003ed4 T owcpa_dec
0000000000003d91 T owcpa_enc
0000000000003afe T owcpa_keypair
00000000000052c6 T poly_lift
00000000000053b4 T poly_mod_3_Phi_n
0000000000005413 T poly_mod_q_Phi_n
0000000000005516 T poly_R2_inv
000000000000524a T poly_Rq_inv
00000000000059ce T poly_Rq_mul
0000000000004e28 T poly_Rq_sum_zero_frombytes
0000000000004dfe T poly_Rq_sum_zero_tobytes
0000000000005464 T poly_Rq_to_S3
0000000000004316 T poly_S3_frombytes
000000000000749a T poly_S3_inv
0000000000004fb8 T poly_S3_mul
00000000000041b6 T poly_S3_tobytes
00000000000048ca T poly_Sq_frombytes
000000

In [9]:
import ctypes
import time
import statistics

ntru = ctypes.CDLL("./libntru2048509.so")

keypair = ntru.keypair
keypair.argtypes = [ctypes.c_void_p, ctypes.c_void_p]
keypair.restype  = ctypes.c_int

encapsulate = ntru.enc
encapsulate.argtypes = [ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p]
encapsulate.restype  = ctypes.c_int

decapsulate = ntru.dec
decapsulate.argtypes = [ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p]
decapsulate.restype  = ctypes.c_int


NTRU_PK, NTRU_SK    = 1138, 1671
NTRU_CT, NTRU_SS    = 1138, 32


def benchmark_keygen(trials=1000):
    times = []
    for _ in range(trials):
        pk = ctypes.create_string_buffer(NTRU_PK)
        sk = ctypes.create_string_buffer(NTRU_SK)
        t0 = time.perf_counter()
        if keypair(pk, sk) != 0:
            raise RuntimeError("keypair failed")
        t1 = time.perf_counter()
        times.append((t1 - t0) * 1000)
    return times

if __name__ == "__main__":
    # Run 1000 trials
    times = benchmark_keygen(1000)
    print(f"Avg keygen: {statistics.mean(times):.3f} ms")
    print(f"Min keygen: {min(times):.3f} ms")
    print(f"Max keygen: {max(times):.3f} ms")
    print(f"Stddev   : {statistics.stdev(times):.3f} ms")


Avg keygen: 28.810 ms
Min keygen: 26.563 ms
Max keygen: 82.073 ms
Stddev   : 4.411 ms


In [10]:
import ctypes
import time
import statistics


ntru = ctypes.CDLL("./libntru2048509.so")


keypair    = ntru.keypair
keypair.argtypes = [ctypes.c_void_p, ctypes.c_void_p]
keypair.restype  = ctypes.c_int

encapsulate = ntru.enc
encapsulate.argtypes = [ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p]
encapsulate.restype  = ctypes.c_int

decapsulate = ntru.dec
decapsulate.argtypes = [ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p]
decapsulate.restype  = ctypes.c_int

NTRU_PK, NTRU_SK = 1138, 1671
NTRU_CT, NTRU_SS = 1138,   32


def functional_test():
    pk = ctypes.create_string_buffer(NTRU_PK)
    sk = ctypes.create_string_buffer(NTRU_SK)
    ct = ctypes.create_string_buffer(NTRU_CT)
    ss = ctypes.create_string_buffer(NTRU_SS)
    ss_dec = ctypes.create_string_buffer(NTRU_SS)

    assert keypair(pk, sk) == 0
    assert encapsulate(ct, ss, pk) == 0
    assert decapsulate(ss_dec, ct, sk) == 0

    print(f"[Functional] NTRU shared secret match: {ss.raw == ss_dec.raw}")


def benchmark(fn, buf_sizes, trials=1000):
    times = []
    for _ in range(trials):
        bufs = [ctypes.create_string_buffer(sz) for sz in buf_sizes]
        start = time.perf_counter()
        assert fn(*bufs) == 0
        end = time.perf_counter()
        times.append((end - start)*1000)
    return times

def print_stats(label, times):
    print(f"[Benchmark] {label}: avg={statistics.mean(times):.3f} ms, "
          f"min={min(times):.3f} ms, max={max(times):.3f} ms, std={statistics.stdev(times):.3f} ms")

if __name__ == "__main__":
    functional_test()
    trials = 1000
    print_stats("NTRU KeyGen",    benchmark(keypair,    [NTRU_PK, NTRU_SK],     trials))
    print_stats("NTRU Encapsulate",benchmark(encapsulate, [NTRU_CT, NTRU_SS, NTRU_PK], trials))
    print_stats("NTRU Decapsulate",benchmark(decapsulate, [NTRU_SS, NTRU_CT, NTRU_SK], trials))


[Functional] NTRU shared secret match: True
[Benchmark] NTRU KeyGen: avg=29.361 ms, min=26.563 ms, max=44.908 ms, std=4.404 ms
[Benchmark] NTRU Encapsulate: avg=0.436 ms, min=0.405 ms, max=1.136 ms, std=0.076 ms
[Benchmark] NTRU Decapsulate: avg=0.915 ms, min=0.871 ms, max=1.499 ms, std=0.112 ms


In [11]:
!pip install psutil


In [12]:
import ctypes
import psutil
import time

ntru = ctypes.CDLL("./libntru2048509.so")


keypair = ntru.keypair
keypair.argtypes = [ctypes.c_void_p, ctypes.c_void_p]
keypair.restype  = ctypes.c_int

NTRU_PK = 1138
NTRU_SK = 1671


proc = psutil.Process()

pk = ctypes.create_string_buffer(NTRU_PK)
sk = ctypes.create_string_buffer(NTRU_SK)
keypair(pk, sk)
time.sleep(0.1)

mem_before = proc.memory_info().rss

pk = ctypes.create_string_buffer(NTRU_PK)
sk = ctypes.create_string_buffer(NTRU_SK)
keypair(pk, sk)


mem_after = proc.memory_info().rss

print(f"NTRU-HPS-2048_509 KeyGen Memory (RSS):")
print(f"  Before: {mem_before/1024:.1f} KB")
print(f"  After:  {mem_after/1024:.1f} KB")
print(f"  Delta:  {(mem_after-mem_before)/1024:.1f} KB")


NTRU-HPS-2048_509 KeyGen Memory (RSS):
  Before: 102980.0 KB
  After:  102980.0 KB
  Delta:  0.0 KB


In [13]:
!sudo apt-get update
!sudo apt-get install time


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [14]:
!which time


/usr/bin/time


In [15]:
%%shell
/usr/bin/time -f "NTRU-509 KeyGen PeakRSS: %M KB" python3 - << 'PYCODE'
import ctypes

ntru = ctypes.CDLL("./libntru2048509.so")
kp   = ntru.keypair
kp.argtypes = [ctypes.c_void_p, ctypes.c_void_p]


pk = ctypes.create_string_buffer(1138)
sk = ctypes.create_string_buffer(1671)
kp(pk, sk)
PYCODE


NTRU-509 KeyGen PeakRSS: 13756 KB


In [16]:
import ctypes
import time


ntru = ctypes.CDLL("./libntru2048509.so")


keypair    = ntru.keypair
keypair.argtypes = [ctypes.c_void_p, ctypes.c_void_p]
keypair.restype  = ctypes.c_int

encapsulate = ntru.enc
encapsulate.argtypes = [ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p]
encapsulate.restype  = ctypes.c_int

decapsulate = ntru.dec
decapsulate.argtypes = [ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p]
decapsulate.restype  = ctypes.c_int


pk = ctypes.create_string_buffer(699)
sk = ctypes.create_string_buffer(935)
ct = ctypes.create_string_buffer(699)
ss = ctypes.create_string_buffer(32)

start = time.perf_counter_ns()
keypair(pk, sk)
keygen_time = time.perf_counter_ns() - start

start = time.perf_counter_ns()
encapsulate(ct, ss, pk)
enc_time = time.perf_counter_ns() - start


start = time.perf_counter_ns()
decapsulate(ss, ct, sk)
dec_time = time.perf_counter_ns() - start

keygen_time //= 1000
enc_time    //= 1000
dec_time    //= 1000


print("KeyGen:", keygen_time, "µs")
print("Encapsulation:", enc_time, "µs")
print("Decapsulation:", dec_time, "µs")


KeyGen: 29737 µs
Encapsulation: 507 µs
Decapsulation: 947 µs


In [17]:
!lscpu


Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4400.26
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt
                           aes xsave avx f16c rdrand hypervisor 

In [18]:
!free -h


               total        used        free      shared  buff/cache   available
Mem:            12Gi       774Mi       7.3Gi       1.0Mi       4.7Gi        11Gi
Swap:             0B          0B          0B


In [19]:
!pip install pyRAPL


In [ ]:

# following code will work on laptop and Raspberry Pi only, not on collab.
import ctypes
import pyRAPL
import sys

pyRAPL.setup()
meter = pyRAPL.Measurement('ntru_full_kem')


ntru = ctypes.CDLL("./libntru2048509.so")
kp = ntru.keypair
enc = ntru.enc
dec = ntru.dec

kp.argtypes = [ctypes.c_void_p, ctypes.c_void_p]
enc.argtypes = [ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p]
dec.argtypes = [ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p]


PK_BYTES = 1138
SK_BYTES = 1671
CT_BYTES = 1138
SS_BYTES = 32

pk = ctypes.create_string_buffer(PK_BYTES)
sk = ctypes.create_string_buffer(SK_BYTES)
ct = ctypes.create_string_buffer(CT_BYTES)
ss = ctypes.create_string_buffer(SS_BYTES)

meter.begin()
kp(pk, sk)
enc(ct, ss, pk)
dec(ss, ct, sk)
meter.end()

print("NTRU-509 full KEM energy (µJ):", meter.result.pkg)


In [ ]:
# ntru_server.py
import socket

HOST, PORT = 'localhost', 7000
sock = socket.socket()
sock.bind((HOST, PORT))
sock.listen(1)
conn, _ = sock.accept()

data = conn.recv(8192)
print("NTRU-509 ciphertext bytes received:", len(data))
conn.close()
sock.close()
